<a href="https://colab.research.google.com/github/mk9165/FlyAI/blob/main/KeyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/MaartenGr/KeyBERT

https://github.com/bab2min/kiwipiepy

In [ ]:
%%capture
!pip install KeyBERT
!pip install kiwipiepy

In [ ]:
from keybert import KeyBERT
from transformers import BertModel

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

In [ ]:
kiwi = Kiwi(num_workers=0, load_default_dict=True, integrate_allomorph=True, model_type='sbg', typos='basic')
stopwords = Stopwords()

문장 전체

In [ ]:
def sentence(text):
  result = kiwi.tokenize(text, normalize_coda=True, stopwords=stopwords)  
  result = kiwi.join(result)
  return result

명사만

In [ ]:
def noun(text):
  results = [] 
  result = kiwi.tokenize(text, normalize_coda=True, stopwords=stopwords)
  for i in range(len(result)):
    token, tag, _, _ = result[i]
    if len(token) != 1 and tag.startswith('N') or tag.startswith('SL'):
      results.append(token)
  results = ' '.join(results)
  return results

Topic 추출

In [ ]:
def predict(doc, method=noun):
  doc = method(doc)
  doc = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1,3), stop_words=None, top_n=3, use_mmr=True, diversity=0.6)
  return doc

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FlyAI/project/summarization/summarization_val.csv')

In [ ]:
sample = np.random.choice(5000, size=5, replace=False)

In [ ]:
for i in sample:
  print('<sentence>', predict(df['text'][i], sentence))
  print('<noun>', predict(df['text'][i]))
  print('-'*50)

<sentence> [('집네 노트북 챙기', 0.8483), ('배달밖에 비대면 준비려면', 0.5526), ('학원', 0.4505)]
<noun> [('토욜 노트북 학원', 0.893), ('카페 배달', 0.5502), ('대면 준비 토욜', 0.4858)]
--------------------------------------------------
<sentence> [('오늘 운동 있을까', 0.7838), ('저녁 라면', 0.7208), ('얼큰하 라면지 이모티콘', 0.2978)]
<noun> [('오홍 저녁 라면', 0.9479), ('라면 라면', 0.6195), ('오늘 운동', 0.6053)]
--------------------------------------------------
<sentence> [('마싯는데 끓이기만하묭되자누 냄비', 0.795), ('다음 빌리', 0.3701), ('계곡', 0.3387)]
<noun> [('끓이기만하묭되자누 냄비 밥솥', 0.958), ('다음', 0.5981), ('계곡', 0.3378)]
--------------------------------------------------
<sentence> [('간당 무슨 식빵', 0.7963), ('치즈랑 블베 맛있', 0.5548), ('블베 맛있 버터', 0.4495)]
<noun> [('베이컨 치즈 필요', 0.915), ('블럭 제빵 식빵', 0.7115), ('버터', 0.6144)]
--------------------------------------------------
<sentence> [('머리 갑자기 돈다', 0.8094), ('개씩', 0.4974), ('초콜릿', 0.448)]
<noun> [('초콜릿 새벽 과제', 0.9207), ('새벽 과제 그것', 0.513), ('머리', 0.4684)]
--------------------------------------------------
